# Emotion Detection using Speech analysis

So here we are analyzing the famous RAVDESS dataset consisting of audio samples from 12 male and 12 female voice artists.

Let's start by importing some essential libraries

In [ ]:
#importing

import os
import glob
from tqdm.autonotebook import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from python_speech_features import mfcc , logfbank
import librosa as lr
import os, glob, pickle
import librosa
from scipy import signal
import noisereduce as nr
from glob import glob
import librosa
get_ipython().magic('matplotlib inline')
import soundfile
from tensorflow.keras.layers import Conv2D,MaxPool2D, Flatten, LSTM
from keras.layers import Dropout,Dense,TimeDistributed
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import speech_recognition as sr


In [ ]:
!pip install SpeechRecognition python_speech_features noisereduce librosa

So we will now get the .wav files and store into **list of files**

In [ ]:
os.listdir(path='/kaggle/input/ravdess-emotional-speech-audio')

def getListOfFiles(dirName):
    listOfFile=os.listdir(dirName)
    allFiles=list()
    for entry in listOfFile:
        fullPath=os.path.join(dirName, entry)
        if os.path.isdir(fullPath):
            allFiles=allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
    return allFiles

dirName = '/kaggle/input/ravdess-emotional-speech-audio'
listOfFiles = getListOfFiles(dirName)
len(listOfFiles)


Here we have 2880 audio samples. 

Now we can check what speech to text gives. We will be using google's speech recognition library to check..

# Sample file

In [ ]:

listOfFiles[0]

In [ ]:

r=sr.Recognizer()
#len(listOfFiles)
for file in range(0 , 4 , 1):
    with sr.AudioFile(listOfFiles[file]) as source:
        audio = r.listen(source)
        try:
            text = r.recognize_google(audio)
            print(text)
        except:
            print('error')


In [ ]:
def envelope(y , rate, threshold):
    mask=[]
    
    # Here we have signal y
    # we want to get absolute values for the signal samples
    print(f"before : {y}")
    y=pd.Series(y).apply(np.abs)
    print(f"after : {y}")
    y_mean = y.rolling(window=int(rate/10) , min_periods=1 , center = True).mean()
    print(f"y mean after rolling : {y_mean}")
    for mean in y_mean:
        if mean>threshold:
            mask.append(True)
        else:
            mask.append(False)
    return mask


In [ ]:
for file in range(0 , 4 , 1):
    audio , sfreq = lr.load(listOfFiles[file])
    time = np.arange(0 , len(audio)) / sfreq
    fig ,ax = plt.subplots()
    ax.plot(time , audio)
    ax.set(xlabel = 'Time (s)' , ylabel = 'Sound Amplitude')
    plt.show()
#len(listOfFiles)
#PLOT THE SEPCTOGRAM

for file in range(0 , 4 , 1):
    sample_rate , samples = wavfile.read(listOfFiles[file])
    frequencies , times, spectrogram = signal.spectrogram(samples, sample_rate)
    plt.pcolormesh(times, frequencies, spectrogram)
    plt.imshow(spectrogram)
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.show()


In [ ]:
def plot_signals(signals):
    fig , axes = plt.subplots(nrows=2, ncols=5,sharex =False , sharey=True, figsize=(20,5))
    fig.suptitle('Time Series' , size=16)
    i=0
    for x in range(2):
        for y in range(5):
            axes[x,y].set_title(list(signals.keys())[i])
            axes[x,y].plot(list(signals.values())[i])
            axes[x,y].get_xaxis().set_visible(False)
            axes[x,y].get_yaxis().set_visible(False)
            i +=1

def plot_fft(fft):
    fig , axes = plt.subplots(nrows=2, ncols=5,sharex =False , sharey=True, figsize=(20,5))
    fig.suptitle('Fourier Transform' , size=16)
    i=0
    for x in range(2):
        for y in range(5):
            data = list(fft.values())[i]
            Y,freq = data[0] , data[1]
            axes[x,y].set_title(list(fft.keys())[i])
            axes[x,y].plot(freq , Y)
            axes[x,y].get_xaxis().set_visible(False)
            axes[x,y].get_yaxis().set_visible(False)
            i +=1

def plot_fbank(fbank):
    fig , axes = plt.subplots(nrows=2, ncols=5,sharex =False , sharey=True, figsize=(20,5))
    fig.suptitle('Filter Bank Coefficients' , size=16)
    i=0
    for x in range(2):
        for y in range(5):
            axes[x,y].set_title(list(fbank.keys())[i])
            axes[x,y].imshow(list(fbank.values())[i],cmap='hot', interpolation = 'nearest')
            axes[x,y].get_xaxis().set_visible(False)
            axes[x,y].get_yaxis().set_visible(False)
            i +=1

def plot_mfccs(mfccs):
     fig , axes = plt.subplots(nrows=2, ncols=5,sharex =False , sharey=True, figsize=(20,5))
     fig.suptitle('Mel Frequency Capstrum Coefficients' , size=16)
     i=0
     for x in range(2):
         for y in range(5):
             axes[x,y].set_title(list(mfccs.keys())[i])
             axes[x,y].imshow(list(mfccs.values())[i],
             cmap='hot', interpolation = 'nearest')
             axes[x,y].get_xaxis().set_visible(False)
             axes[x,y].get_yaxis().set_visible(False)
             i +=1
def calc_fft(y,rate):
    # calculates FFT - Fast fourier transform of signal
    n = len(y)
    freq = np.fft.rfftfreq(n , d= 1/rate)
    Y= abs(np.fft.rfft(y)/n)
    #print(type(Y))
    #print(type(freq))
    return(Y,freq)


In [61]:
import matplotlib.pyplot as plt
from scipy.io import wavfile as wav
from scipy.fftpack import fft
import numpy as np
import sys

'''
for file in range(0 , len(listOfFiles) , 1):
    rate, data = wav.read(listOfFiles[file])
    fft_out = fft(data)
    %matplotlib inline
    plt.plot(data, np.abs(fft_out))
    plt.show()
'''
signals={}
fft={}
fbank={}
mfccs={}
# load data
for file in range(0 , 2 , 1):
# rate, data = wavfile.read(listOfFiles[file])
    print("Files are being given as input are .wav files")
    
    
    signal,rate =librosa.load(listOfFiles[file] , sr=44100)
    print(f"File {file}\nAnalysis length : {len(signal)} sampling rate : {rate}\nsignal numpy array : {signal}\n--\n")
    #print(signal)
    
    
    mask = envelope(signal , rate , 0.0005)
    print(f"mask is {type(mask)} of signal without the empty vocals or audio using rolling\n If above thresh then considered otherwise not.")
    print("We are only focusing on the data we want to use further")
    
    
    signals[file] = signal
    fft[file] = calc_fft(signal , rate)

    bank = logfbank(signal[:rate] , rate , nfilt = 26, nfft = 1103).T
    fbank[file] = bank
    mel = mfcc(signal[:rate] , rate , numcep =13 , nfilt = 26 , nfft=1103).T
    mfccs[file]=mel
    
    '''
    Mel-scaled filter banks and Mel-Frequency Cepstrum Coefficients
    '''

'''
plot_signals(signals)
plt.show()
plot_fft(fft)
plt.show()
plot_fbank(fbank)
plt.show()
plot_mfccs(mfccs)
plt.show()
'''
print(f"signal : {signal}")
print(f"fft : {fft}")
print(f"fbank : {fbank}")
print(f"mfccs : {mfccs}")

Files are being given as input are .wav files
File 0
Analysis length : 170691 sampling rate : 44100
signal numpy array : [0. 0. 0. ... 0. 0. 0.]
--

mask is <class 'list'> of signal without the empty vocals or audio using rolling
 If above thresh then considered otherwise not.
We are only focusing on the data we want to use further
Files are being given as input are .wav files
File 1
Analysis length : 186878 sampling rate : 44100
signal numpy array : [-2.4816152e-06  3.4122331e-06 -4.7799690e-06 ... -8.4949170e-07
  1.2130616e-07  0.0000000e+00]
--

mask is <class 'list'> of signal without the empty vocals or audio using rolling
 If above thresh then considered otherwise not.
We are only focusing on the data we want to use further
signal : [-2.4816152e-06  3.4122331e-06 -4.7799690e-06 ... -8.4949170e-07
  1.2130616e-07  0.0000000e+00]
fft : {0: (array([7.64873009e-08, 2.06147925e-07, 2.65015192e-07, ...,
       3.06306742e-09, 3.63350070e-10, 4.55816661e-09]), array([0.00000000e+00, 2.

In [62]:
#Now Cleaning Step is Performed where:
#DOWN SAMPLING OF AUDIO FILES IS DONE  AND PUT MASK OVER IT AND DIRECT INTO CLEAN FOLDER
#MASK IS TO REMOVE UNNECESSARY EMPTY VOIVES AROUND THE MAIN AUDIO VOICE 
def envelope(y , rate, threshold):
    mask=[]
    y=pd.Series(y).apply(np.abs)
    y_mean = y.rolling(window=int(rate/10) ,  min_periods=1 , center = True).mean()
    for mean in y_mean:
        if mean>threshold:
            mask.append(True)
        else:
            mask.append(False)
    return mask

In [63]:
#The clean Audio Files are redirected to Clean Audio Folder Directory 
import glob,pickle

print("We have",len(listOfFiles)," files")
print(listOfFiles[0])

print("Iterating through the .wav samples one by one")

#glob.glob(r'\kaggle\input\\**\\*.wav')
for file in tqdm(listOfFiles):
    print(file)
    file_name = os.path.basename(file)
    signal , rate = librosa.load(file, sr=16000)
    mask = envelope(signal,rate, 0.0005)
    wavfile.write(filename=r''+str(file_name), rate=rate,data=signal[mask])

We have 2880  files
/kaggle/input/ravdess-emotional-speech-audio/Actor_22/03-01-05-02-01-01-22.wav
Iterating through the .wav samples one by one


/kaggle/input/ravdess-emotional-speech-audio/Actor_22/03-01-05-02-01-01-22.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_22/03-01-05-02-02-01-22.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_22/03-01-04-01-02-02-22.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_22/03-01-07-02-01-02-22.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_22/03-01-08-02-02-01-22.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_22/03-01-04-02-02-01-22.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_22/03-01-06-01-01-02-22.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_22/03-01-05-01-01-02-22.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_22/03-01-05-01-01-01-22.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_22/03-01-07-02-02-02-22.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_22/03-01-04-01-01-01-22.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_22/03-01-05-01-02-02-22.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_2

/kaggle/input/ravdess-emotional-speech-audio/Actor_04/03-01-02-02-02-02-04.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_04/03-01-04-01-01-02-04.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_04/03-01-06-02-02-02-04.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_04/03-01-02-02-02-01-04.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_04/03-01-04-01-02-01-04.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_04/03-01-07-01-01-02-04.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_04/03-01-08-02-01-02-04.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_04/03-01-05-01-02-02-04.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_04/03-01-08-01-02-02-04.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_04/03-01-01-01-02-01-04.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_04/03-01-06-02-02-01-04.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_04/03-01-05-02-02-01-04.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_0

/kaggle/input/ravdess-emotional-speech-audio/audio_speech_actors_01-24/Actor_22/03-01-02-02-02-02-22.wav
/kaggle/input/ravdess-emotional-speech-audio/audio_speech_actors_01-24/Actor_22/03-01-08-01-01-02-22.wav
/kaggle/input/ravdess-emotional-speech-audio/audio_speech_actors_01-24/Actor_22/03-01-03-01-01-01-22.wav
/kaggle/input/ravdess-emotional-speech-audio/audio_speech_actors_01-24/Actor_22/03-01-05-01-02-01-22.wav
/kaggle/input/ravdess-emotional-speech-audio/audio_speech_actors_01-24/Actor_22/03-01-02-02-01-01-22.wav
/kaggle/input/ravdess-emotional-speech-audio/audio_speech_actors_01-24/Actor_22/03-01-04-01-02-01-22.wav
/kaggle/input/ravdess-emotional-speech-audio/audio_speech_actors_01-24/Actor_22/03-01-03-02-01-01-22.wav
/kaggle/input/ravdess-emotional-speech-audio/audio_speech_actors_01-24/Actor_22/03-01-01-01-02-02-22.wav
/kaggle/input/ravdess-emotional-speech-audio/audio_speech_actors_01-24/Actor_22/03-01-02-02-01-02-22.wav
/kaggle/input/ravdess-emotional-speech-audio/audio_spee

/kaggle/input/ravdess-emotional-speech-audio/audio_speech_actors_01-24/Actor_04/03-01-08-02-01-01-04.wav
/kaggle/input/ravdess-emotional-speech-audio/audio_speech_actors_01-24/Actor_04/03-01-02-01-02-01-04.wav
/kaggle/input/ravdess-emotional-speech-audio/audio_speech_actors_01-24/Actor_04/03-01-06-01-02-02-04.wav
/kaggle/input/ravdess-emotional-speech-audio/audio_speech_actors_01-24/Actor_04/03-01-01-01-01-01-04.wav
/kaggle/input/ravdess-emotional-speech-audio/audio_speech_actors_01-24/Actor_04/03-01-02-02-02-02-04.wav
/kaggle/input/ravdess-emotional-speech-audio/audio_speech_actors_01-24/Actor_04/03-01-04-01-01-02-04.wav
/kaggle/input/ravdess-emotional-speech-audio/audio_speech_actors_01-24/Actor_04/03-01-06-02-02-02-04.wav
/kaggle/input/ravdess-emotional-speech-audio/audio_speech_actors_01-24/Actor_04/03-01-02-02-02-01-04.wav
/kaggle/input/ravdess-emotional-speech-audio/audio_speech_actors_01-24/Actor_04/03-01-04-01-02-01-04.wav
/kaggle/input/ravdess-emotional-speech-audio/audio_spee

...

/kaggle/input/ravdess-emotional-speech-audio/Actor_21/03-01-06-01-02-02-21.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_21/03-01-05-01-02-02-21.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_21/03-01-07-01-02-02-21.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_21/03-01-06-02-02-02-21.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_21/03-01-04-01-01-01-21.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_21/03-01-07-01-02-01-21.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_21/03-01-05-02-01-02-21.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_21/03-01-03-02-02-02-21.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_21/03-01-02-01-02-01-21.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_21/03-01-08-01-02-01-21.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_21/03-01-03-01-01-01-21.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_21/03-01-04-02-02-02-21.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_2

/kaggle/input/ravdess-emotional-speech-audio/Actor_10/03-01-03-01-02-01-10.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_10/03-01-02-02-02-02-10.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_10/03-01-02-02-01-01-10.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_10/03-01-06-02-02-01-10.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_10/03-01-08-01-02-02-10.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_10/03-01-03-02-01-02-10.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_10/03-01-03-01-01-02-10.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_10/03-01-06-01-02-01-10.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_10/03-01-02-02-02-01-10.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_10/03-01-07-02-01-01-10.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_10/03-01-08-02-01-02-10.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_10/03-01-05-01-01-02-10.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_1

/kaggle/input/ravdess-emotional-speech-audio/Actor_14/03-01-06-01-02-01-14.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_14/03-01-02-01-01-02-14.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_14/03-01-04-02-02-02-14.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_14/03-01-06-02-02-01-14.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_14/03-01-05-02-01-02-14.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_14/03-01-05-01-02-01-14.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_09/03-01-06-02-01-02-09.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_09/03-01-07-02-01-02-09.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_09/03-01-04-01-02-01-09.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_09/03-01-04-02-02-01-09.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_09/03-01-03-01-01-01-09.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_09/03-01-05-02-02-01-09.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_0

/kaggle/input/ravdess-emotional-speech-audio/Actor_15/03-01-04-01-02-02-15.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_15/03-01-02-01-01-02-15.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_15/03-01-06-01-02-01-15.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_15/03-01-07-02-01-02-15.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_15/03-01-08-02-02-02-15.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_15/03-01-04-01-01-02-15.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_15/03-01-08-01-01-02-15.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_15/03-01-02-01-01-01-15.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_15/03-01-03-02-02-02-15.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_15/03-01-05-02-02-01-15.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_15/03-01-05-02-01-02-15.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_15/03-01-05-01-02-01-15.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_1

/kaggle/input/ravdess-emotional-speech-audio/Actor_17/03-01-08-01-02-01-17.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_17/03-01-07-02-01-01-17.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_17/03-01-06-02-02-02-17.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_17/03-01-01-01-01-02-17.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_17/03-01-06-01-02-01-17.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_17/03-01-03-02-01-01-17.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_17/03-01-06-02-02-01-17.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_17/03-01-08-01-02-02-17.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_17/03-01-01-01-02-02-17.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_17/03-01-05-01-01-01-17.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_17/03-01-03-02-01-02-17.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_17/03-01-05-01-01-02-17.wav
/kaggle/input/ravdess-emotional-speech-audio/Actor_1

In [64]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
        result=np.hstack((result, mel))
    return result

In [65]:
#Emotions in the RAVDESS dataset to be classified Audio Files based on . 
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}
#These are the emotions User wants to observe more :
observed_emotions=['calm', 'happy', 'fearful', 'disgust']


In [71]:
#Load the data and extract features for each sound file
from glob import glob
import os
import glob

list_of_cleaned_files = glob.glob('/kaggle/working/*.wav')

print(len(list_of_cleaned_files))
print(list_of_cleaned_files[0])
print(list_of_cleaned_files[1])


list_of_cleaned_files[0]
def load_data(test_size=0.33):
    x,y=[],[]
    answer = 0
    for file in list_of_cleaned_files:
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            answer += 1
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append([emotion,file_name])
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

1440
/kaggle/working/03-01-07-02-02-01-04.wav
/kaggle/working/03-01-04-01-01-01-12.wav


# Training the MLP model

In [72]:
#Split the dataset
import librosa
import numpy as np
x_train,x_test,y_trai,y_tes=load_data(test_size=0.25)
print(np.shape(x_train),np.shape(x_test), np.shape(y_trai),np.shape(y_tes))
y_test_map = np.array(y_tes).T
y_test = y_test_map[0]
test_filename = y_test_map[1]
y_train_map = np.array(y_trai).T
y_train = y_train_map[0]
train_filename = y_train_map[1]
print(np.shape(y_train),np.shape(y_test))
print(*test_filename,sep="\n")

#Get the shape of the training and testing datasets
# print((x_train.shape[0], x_test.shape[0]))
print((x_train[0], x_test[0]))
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

# Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

#Train the model
model.fit(x_train,y_train)



(576, 180) (192, 180) (576, 2) (192, 2)
(576,) (192,)
03-01-03-02-01-02-15.wav
03-01-02-02-01-01-23.wav
03-01-03-02-02-01-22.wav
03-01-07-02-02-02-14.wav
03-01-03-02-02-02-02.wav
03-01-07-02-02-01-24.wav
03-01-07-01-02-01-22.wav
03-01-07-02-02-02-12.wav
03-01-02-02-01-02-16.wav
03-01-03-02-02-01-05.wav
03-01-07-02-02-01-06.wav
03-01-02-01-02-02-23.wav
03-01-03-01-01-01-07.wav
03-01-07-01-01-01-21.wav
03-01-03-02-01-02-09.wav
03-01-03-02-01-01-21.wav
03-01-07-01-02-01-04.wav
03-01-07-02-01-01-23.wav
03-01-03-01-01-01-03.wav
03-01-06-01-01-02-17.wav
03-01-03-02-01-02-06.wav
03-01-07-01-01-02-08.wav
03-01-07-01-01-02-02.wav
03-01-06-02-02-02-14.wav
03-01-03-01-02-02-20.wav
03-01-07-02-01-01-02.wav
03-01-03-02-01-02-23.wav
03-01-02-01-02-02-17.wav
03-01-07-02-01-01-05.wav
03-01-07-01-02-01-16.wav
03-01-03-02-01-02-02.wav
03-01-07-01-02-01-21.wav
03-01-03-01-01-02-11.wav
03-01-02-01-02-02-01.wav
03-01-02-01-01-02-20.wav
03-01-06-02-02-01-02.wav
03-01-06-02-01-02-18.wav
03-01-02-01-01-01-15.

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

# **Predictions**

In [73]:
#SAVING THE MODEL
import pickle
# Save the Modle to file in the current working directory
#For any new testing data other than the data in dataset

Pkl_Filename = r"\kaggle\working\Emotion_Voice_Detection_Model.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(model, file)

# Load the Model back from file
with open(Pkl_Filename, 'rb') as file:  
    Emotion_Voice_Detection_Model = pickle.load(file)

Emotion_Voice_Detection_Model

#predicting :
y_pred=Emotion_Voice_Detection_Model.predict(x_test)
y_pred

#Store the Prediction probabilities into CSV file 
import numpy as np
import pandas as pd
y_pred1 = pd.DataFrame(y_pred, columns=['predictions'])
y_pred1['file_names'] = test_filename
print(y_pred1)
y_pred1.to_csv(r"\kaggle\working\predictionfinal.csv")

    predictions                file_names
0         happy  03-01-03-02-01-02-15.wav
1          calm  03-01-02-02-01-01-23.wav
2       disgust  03-01-03-02-02-01-22.wav
3         happy  03-01-07-02-02-02-14.wav
4         happy  03-01-03-02-02-02-02.wav
..          ...                       ...
187     fearful  03-01-03-02-01-02-03.wav
188     disgust  03-01-06-02-02-02-09.wav
189       happy  03-01-03-01-02-01-20.wav
190        calm  03-01-07-02-02-01-13.wav
191       happy  03-01-03-02-01-01-01.wav

[192 rows x 2 columns]


# Emotion prediction for recorded audio

In [ ]:
#RECORDED USING MICROPHONE:
import pyaudio
import wave

CHUNK = 1024 
FORMAT = pyaudio.paInt16 #paInt8
CHANNELS = 2 
RATE = 44100 #sample rate
RECORD_SECONDS = 4
WAVE_OUTPUT_FILENAME = "output10.wav"

WAVE_OUTPUT_FILENAME = listOfFiles[0]



p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK) #buffer

print("* recording")
frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data) # 2 bytes(16 bits) per channel

print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

#The file 'output10.wav' in the next cell is the file that was recorded live using the code :
data, sampling_rate = librosa.load('output10.wav')
%matplotlib inline
import os
import pandas as pd
import librosa.display
import glob 

plt.figure(figsize=(15, 5))
librosa.display.waveplot(data, sr=sampling_rate)

## Appying extract_feature function on random file and then loading model to predict the result 
file = 'output10.wav'
# data , sr = librosa.load(file)
# data = np.array(data)
ans =[]
new_feature ,labels  = extract_feature(file, mfcc=True, chroma=True, mel=True)
ans.append(new_feature)
ans = np.array(ans)
# data.shape

Emotion_Voice_Detection_Model.predict([ans])